## Food2Fork API ##
www.food2fork.com/about/api

In [49]:
# API access
import requests
import urllib2
from urllib import urlencode
import json
from bs4 import BeautifulSoup

# Plotting
from matplotlib import pyplot as plt

# Parsing ingredient lists
import re
import nltk
import itertools
from itertools import compress

In [50]:
%matplotlib inline

### API credentials ###

In [51]:
def loadCredentials():
    filename = 'secrets.txt'
    for line in open(filename).readlines():
        if "API" in line:    
            api_key = line.split(": ")[1].translate(None,'\n')
        
    return api_key

In [52]:
# Load the API Key
_API_KEY = loadCredentials()

# Food2Fork links
_URL_API = "http://food2fork.com/api/"
_URL_SEARCH = _URL_API + 'search?'
_URL_GET    = _URL_API + 'get?'
_HEADER = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}

### Search the Food2Fork API ###

In [53]:
def search(query, page=1, count=1):   
    """Return a list of recipes from the Food2Fork.com database"""
    
    assert(0 < count <= 30), 'max 30 results per call, min 1' #https://github.com/davebshow/food2forkclient/
        
    # Format the request URL
    query_params = {"key":_API_KEY,"q":query, "page":page, "count":count}            
    api_request = _URL_SEARCH + urlencode(query_params)    

    # Make the request
    request  = urllib2.Request(api_request, headers=_HEADER)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)  
    
    return json_obj['recipes']

In [54]:
# Search for cookie recipes
recipes = search('Chocolate chip cookies',count=5)
print(recipes[0]['image_url'])
recipes[0]

http://static.food2fork.com/508714d505.jpg


{u'f2f_url': u'http://food2fork.com/view/6868',
 u'image_url': u'http://static.food2fork.com/508714d505.jpg',
 u'publisher': u'All Recipes',
 u'publisher_url': u'http://allrecipes.com',
 u'recipe_id': u'6868',
 u'social_rank': 99.99999999999996,
 u'source_url': u'http://allrecipes.com/Recipe/Chewy-Chocolate-Chip-Oatmeal-Cookies/Detail.aspx',
 u'title': u'Chewy Chocolate Chip Oatmeal Cookies'}

### Request a specific recipe from the API ###

In [55]:
def getRecipe(recipe_id):
    """Return the recipe specified by *recipe_id* from Food2Fork's database"""

    # Format the request URL
    api_request = _URL_GET + urlencode({"key":_API_KEY, "rId":recipe_id})
    
    # Make the request
    request  = urllib2.Request(api_request, headers=_HEADER)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)  
    
    return json_obj['recipe']

In [56]:
r = getRecipe('35130')
r

{u'f2f_url': u'http://food2fork.com/view/35130',
 u'image_url': u'http://static.food2fork.com/BananasFosterPancakes15000eeed5d6.jpg',
 u'ingredients': [u'1 cup all purpose flour',
  u'1 teaspoon baking powder',
  u'1/4 teaspoon salt',
  u'1/4 teaspoon cinnamon',
  u'1 cup milk',
  u'1 egg',
  u'2 tablespoons brown sugar',
  u'2 tablespoons unsalted butter (melted)',
  u'2 overripe bananas (peeled and mashed)',
  u'2 tablespoons butter',
  u'2 tablespoons brown sugar',
  u'2 dashes of cinnamon',
  u'2 ripe bananas (peeled and sliced)',
  u'2 ounces dark rum\n'],
 u'publisher': u'Closet Cooking',
 u'publisher_url': u'http://closetcooking.com',
 u'recipe_id': u'35130',
 u'social_rank': 38.259222092243164,
 u'source_url': u'http://www.closetcooking.com/2010/05/bananas-foster-pancakes.html',
 u'title': u'Bananas Foster Pancakes'}

### Scrape ingredients from a Food2Fork.com page ###

In [57]:
def getIngredients(json_obj):
    r = getRecipe(json_obj['recipe_id'])
    return [i.strip('\n').encode('ascii', 'ignore') for i in r['ingredients']]

In [58]:
ingredients = getIngredients(recipes[0])
ingredients

['1 cup butter, softened',
 '1 cup packed light brown sugar',
 '1/2 cup white sugar',
 '2 eggs',
 '2 teaspoons vanilla extract',
 '1 1/4 cups all-purpose flour',
 '1/2 teaspoon baking soda',
 '1 teaspoon salt',
 '3 cups quick-cooking oats',
 '1 cup chopped walnuts',
 '1 cup semisweet chocolate chips']

## Parse ingredient amounts using reg exp and NLTK ##

In [302]:
class Ingredient(object):
    
    def __init__(self, kind, amount, units):
        self._kind   = kind # e.g. butter, sugar, etc. (this needs a better variable name than "kind")        
        self._amount = amount # How many of units?
        self._units = units   # Measurement units (e.g. cup, tablespoon, pound, etc.)
                
    @property
    def kind(self):
        return self._kind
    
    @property
    def amount(self):
        return self._amount
    
    @property
    def units(self):
        return self._units
    
    def __repr__(self):        
        return repr((self.amount, self.units, self.kind))

In [325]:
def parseIngredients(ingrds):
    
    # There must be a function to do the start:stop indexing I'm kludging here
    expr = '(\d{0,2}|(\d*(/|.)\d*))? (\d/\d)?'
    amounts = [i[re.match(expr,i).start():re.match(expr,i).end()].strip() for i in ingrds]    
    units = [i[re.match(expr,i).end():].strip().split()[0] for i in ingrds]
    kinds = [i[re.match(expr,i).end():].strip().split()[1:] for i in ingrds]
                
    # Remove adjectives from kinds    
    colors = ['white','brown']
    POS = [nltk.pos_tag(kind) for kind in kinds]
    mask_not_vbd = [[('VBD'!=word[1])and('VBN'!=word[1]) for word in sent] for sent in POS]       
    kinds = [list(compress(pos,mask)) for pos,mask in zip(POS,mask_not_vbd)]
    kinds = [' '.join([k[0] for k in tup]) for tup in kinds]
    
    # Check for eggs (kludge, how do you deal with this??)
    for n in xrange(len(units)):
        if units[n]=='egg' or units[n]=='eggs':
            kinds[n] = units[n]
            units[n] = units[n]
            
    parsed_ingredients = []
    for i in xrange(len(amounts)):
        parsed_ingredients.append(Ingredient(kinds[i],amounts[i],units[i]))
        
    return parsed_ingredients

In [324]:
recipes = search("Chocolate chip cookies",count=5)

In [327]:
recipe = recipes[0]
ingredients = getIngredients(recipe)
print(recipe['title'])
print(ingredients)
parsed = parseIngredients(ingredients)
parsed

Chewy Chocolate Chip Oatmeal Cookies
['1 cup butter, softened', '1 cup packed light brown sugar', '1/2 cup white sugar', '2 eggs', '2 teaspoons vanilla extract', '1 1/4 cups all-purpose flour', '1/2 teaspoon baking soda', '1 teaspoon salt', '3 cups quick-cooking oats', '1 cup chopped walnuts', '1 cup semisweet chocolate chips']


[('1', 'cup', 'butter,'),
 ('1', 'cup', 'light brown sugar'),
 ('1/2', 'cup', 'white sugar'),
 ('2', 'eggs', 'eggs'),
 ('2', 'teaspoons', 'vanilla extract'),
 ('1 1/4', 'cups', 'all-purpose flour'),
 ('1/2', 'teaspoon', 'baking soda'),
 ('1', 'teaspoon', 'salt'),
 ('3', 'cups', 'quick-cooking oats'),
 ('1', 'cup', 'walnuts'),
 ('1', 'cup', 'semisweet chocolate chips')]